In [1]:
import time
from open_gopro import WirelessGoPro, Params
import os
import requests
import numpy as np
import datetime
from datetime import timezone
from astral.sun import sun
from astral import LocationInfo
import pytz
import json
import re
import signal
import subprocess
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw
import os
import netifaces as ni

In [ ]:
gopro = WirelessGoPro(sudo_password='ay!MLMSL26y')
gopro.open()

ERROR:root:A message handler raised an exception: Connection terminated: BLE connection terminated unexpectedly..
Traceback (most recent call last):
  File "src/dbus_fast/message_bus.py", line 811, in dbus_fast.message_bus.BaseMessageBus._process_message
  File "/home/pi/.local/lib/python3.9/site-packages/bleak/backends/bluezdbus/manager.py", line 852, in _parse_msg
    on_connected_changed(self_interface["Connected"])
  File "/home/pi/.local/lib/python3.9/site-packages/bleak/backends/bluezdbus/client.py", line 143, in on_connected_changed
    self._disconnected_callback(self)
  File "/home/pi/.local/lib/python3.9/site-packages/open_gopro/ble/adapters/bleak_wrapper.py", line 223, in disconnect_cb
    self._client_cb(args) if self._should_use_client_cb else self._connecting_cb(args)
  File "/home/pi/.local/lib/python3.9/site-packages/open_gopro/gopro_wireless.py", line 565, in _disconnect_handler
    raise GpException.ConnectionTerminated("BLE connection terminated unexpectedly.")
open_

In [ ]:
gopro.http_command.set_keep_alive()
gopro.http_command.load_preset_group(group=Params.PresetGroup.PHOTO)
gopro.http_setting.max_lens_mode.set(Params.MaxLensMode(1))
gopro.http_setting.photo_horizon_leveling.set(Params.HorizonLeveling(2))
gopro.http_setting.photo_field_of_view.set(Params.PhotoFOV(101))
gopro.ble_command.set_shutter(Params.ExposureMode.AUTO, Params.Flatmode.SINGLE_PHOTO, shutter=Params.Toggle.ENABLE)

In [3]:
basepath = '/home/pi/autofs/data/gopro/'
date = datetime.datetime.now()

In [31]:
# Download and delete all of the files from the camera
media_list = [x["n"] for x in gopro.http_command.get_media_list().flatten]
for f in media_list:
    #print('Download ', f)
    epoch_file = gopro.http_command.get_media_info(file=f).data['cre']
    utc_file = datetime.datetime.fromtimestamp(int(epoch_file))
    tstr = utc_file.strftime('%Y%m%d-%H%M%S')
    lpath = basepath+'day/'+tstr[0:8]+'/'
    if not os.path.exists(lpath):
        os.makedirs(lpath)
    lfile = lpath+tstr+'.JPG'
    gopro.http_command.download_file(camera_file=f, local_file=lfile)
    filename=tstr+'.JPG'
    splitup = filename.split("-")
    date = splitup[0]
    t = splitup[1][0:4]
    tformatted = t[0:2] + ":" + t[2:4]
    img = Image.open(lfile)
    draw = ImageDraw.Draw(img)
    draw.text((img.width-220,img.height-150), date, fontcolor, font=fontsmall)
    draw.text((img.width-220,img.height-120), tformatted, fontcolor, font=font)
    img.save(lfile)

    #print('Delete ', f)
    url = "http://"+ip+":8080/gp/gpControl/command/storage/delete?p=" + "/100GOPRO/" + f
    with requests.get(url) as request:
        request.raise_for_status()

KeyError: 'cre'

In [30]:
media_list = [x["n"] for x in gopro.http_command.get_media_list().flatten]

In [25]:
for f in media_list:
    gopro.ble_status.batt_level()
    #url = "http://"+ip+":8080/gp/gpControl/command/storage/delete?p=" + "/100GOPRO/" + f
    #with requests.get(url) as request:
    #    request.raise_for_status()

SyntaxError: invalid syntax (3099832412.py, line 2)

In [27]:
gopro.http_command.webcam_start()

KeyboardInterrupt: 